In [1]:
#wet_site:https://blog.csdn.net/u010223750/article/details/71079036
from __future__ import print_function
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()

/Users/admin/anaconda3/envs/py2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#8代表输入层单元的个数，第一个表示batch_size，第二个参数是time_step
embedding_size=8
batch_size = 3
time_step = 11
#定义隐藏层单元的个数，隐藏层单元个数自行定义
hidden_units = 24
X = np.random.randn(batch_size,time_step,embedding_size)
print(np.shape(X))


(3, 11, 8)


In [3]:
#X[1]代表获取第二个样本，将6以后的词的向量全部变为0，即单词向量为0
second_sample_size = 6
third_sample_size = 7
X[1,second_sample_size:]=0
X[1,third_sample_size:]=0
X_lengths=[time_step,second_sample_size,third_sample_size]
#定义一个占位符用于输入,为batch_size,max_timestep,embedding_size)
input_x = tf.placeholder(shape=(None,None,embedding_size),dtype=tf.float64,name="input_x")

In [4]:
#构建RNN网络
import tensorflow as tf

#可以debug进入源码，查看源码中如何获取time_step
lstm = tf.nn.rnn_cell.LSTMCell(num_units=hidden_units, state_is_tuple=True)
#LSTMCell无法构造相应的输入，构造输入需要dy_rnn,这里貌似不再需要指定时间步
#该网络会有一个输入和最后时间步的状态输出。不使用占位符也可以，可以直接传入数据
dy_rnn_outputs,final_state = tf.nn.dynamic_rnn(lstm,inputs=X,dtype=tf.float64,scope="test2")
#该网络没有定义y的值
#del dy_rnn_outputs

In [5]:
#state的第二个参数包含有h和c，所以shape肯定是[batch_size,hiddent_units]，其中根据rnn_cell的种类不同，
#带有h和c输出的shape运行后的结果会为[2,batch_size,hidden_units].
final_state

LSTMStateTuple(c=<tf.Tensor 'test2/while/Exit_3:0' shape=(3, 24) dtype=float64>, h=<tf.Tensor 'test2/while/Exit_4:0' shape=(3, 24) dtype=float64>)

In [6]:

decoder_logits = tf.contrib.layers.linear(dy_rnn_outputs, 3)
#在运行前先进行初始化
init = tf.global_variables_initializer()
sess.run(init)
#logits_result = sess.run(decoder_logits)
#logits_result

In [7]:
final_state_result = sess.run(final_state)
dy_rnn_outputs_result = sess.run(dy_rnn_outputs)

In [8]:
final_state_result[-1]==dy_rnn_outputs_result

/Users/admin/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [9]:
#会有俩个输出，h和c，shape中的第一位肯定是2，代表h的输出和c的输出
print(np.shape(final_state_result))
#[batch_size,times_step,hidden_units]
print(np.shape(dy_rnn_outputs_result))

(2, 3, 24)
(3, 11, 24)


In [10]:
print("h.shape:",np.shape(final_state_result.h))
print("c.shape:",np.shape(final_state_result.c))
print((final_state_result[0]==final_state_result.c).all())

h.shape: (3, 24)
c.shape: (3, 24)
True
